## **Rule-Based IPA Transformation Experiment**

This notebook evaluates the impact of rule-based phonological corrections and boundary-augmented IPA on two models:

1. Baseline IPA → Text model (original T5-small)

2. Fine-tuned IPA → Text model (trained on CHILDES 3-view corpus)

The purpose is to determine whether:

* rule-based corrections help or hurt

* boundary segmentation improves decoding

* fine-tuning changes model sensitivity to IPA forms

In [ ]:
!pip install -q transformers pandas tqdm jiwer

import torch, re, pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from jiwer import wer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.3 MB/s eta 0:00:00


In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **1. Load IPA-CHILDES Data**.

* Loads 500 examples from the CHILDES validation set for fast evaluation.

* Ensures each sample has an IPA transcription and reference text (`text_ref`).

* This dataset serves as the testbed for comparing rule-based and boundary-augmented transformations.

In [ ]:
# ============================================================
# 1. Load IPA-CHILDES data
# ============================================================
data_path = "/content/drive/MyDrive/Capstone/Corpus/ipa_childes/child_valid.tsv"
df = pd.read_csv(data_path, sep="\t")
if "ipa_transcription" not in df.columns:
    df.columns = ["ipa_transcription", "text_ref"]  # adjust if needed

# keep small sample for speed
df = df.sample(500, random_state=42).reset_index(drop=True)

### **2. Clean & Normalize IPA Input**

What this step does

* Removes formatting artifacts (WORD_BOUNDARY, punctuation).

* Normalizes whitespace.

* Produces a clean IPA string so that all later transformations operate consistently.

This step removes noise that could otherwise produce false differences between conditions.

In [ ]:
# ============================================================
# 2. Clean & prepare IPA
# ============================================================
def clean_ipa(ipa):
    if pd.isna(ipa): return ""
    ipa = str(ipa).replace("WORD_BOUNDARY", "").replace(".", "").replace(",", "")
    ipa = re.sub(r"\s+", " ", ipa).strip()
    return ipa

df["ipa_clean"] = df["ipa_transcription"].apply(clean_ipa)

### **3. Create Boundary-Augmented & Rule-Based IPA Variants**

This section generates three different IPA inputs:

* ipa_norules - Clean IPA with no modifications
* ipa_rules - IPA after applying phonological correction rules
* ipa_boundaries - IPA with inserted boundary markers `

These represent the three transformation strategies under evaluation.

Each one tests a different hypothesis:

* Do rules improve decoding?

* Do segmentation cues help the model?

* Does the model degrade when IPA is altered?

In [ ]:
# ============================================================
# 3. Rule-based correction + boundary variant
# ============================================================
def approximate_boundaries(ipa_seq):
    tokens = ipa_seq.split()
    out = []
    for i, tok in enumerate(tokens[:-1]):
        nxt = tokens[i+1]
        out.append(tok)
        if tok in ["a","e","i","o","u","æ","ʌ","ɪ","ʊ","n","m"] and nxt in ["p","t","k"]:
            out.append("|")
    out.append(tokens[-1])
    return " ".join(out)

def correct_phonemes(ipa_seq):
    seq = ipa_seq
    if re.search(r"(æ|ɪ|ʌ|ɑ|oʊ|u)\b", seq):
        seq += " t"
    seq = re.sub(r"b n", "b ə n", seq)
    seq = re.sub(r"n n", "n ə n", seq)
    seq = re.sub(r"ɑ\b", "ɑ ɹ", seq)
    seq = re.sub(r"ɔ\b", "ɔ l", seq)
    if seq.startswith("p"): seq = "s " + seq
    if seq.endswith(("o", "oʊ")): seq += " m"
    return re.sub(r"\s+", " ", seq).strip()

df["ipa_segmented"] = df["ipa_clean"].apply(approximate_boundaries)
df["ipa_corrected"] = df["ipa_segmented"].apply(correct_phonemes)

# Collapse for T5
df["ipa_norules"] = df["ipa_clean"].str.replace(" ", "")
df["ipa_rules"] = df["ipa_corrected"].str.replace(" ", "").str.replace("|", "")
df["ipa_boundaries"] = df["ipa_corrected"].str.replace(" ", "")  # keeps '|'

### **4. Load the Original (Baseline) T5 Model**

The original IPA→Text model is loaded to measure:

* baseline performance

* sensitivity to rule-based and boundary-augmented IPA

* the effectiveness of your later fine-tuning

This provides the initial WER benchmark.

In [ ]:
# ============================================================
# 4. Load your fine-tuned T5 model
# ============================================================
model_id = "zanegraper/t5-small-ipa-phoneme-to-text"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

def decode_ipa(ipa_seq):
    inputs = tokenizer(ipa_seq, return_tensors="pt", padding=True, truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=80)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

### **5. Decode All Three IPA Conditions**

For each of the 500 CHILDES examples:

* Feeds three IPA variants to the model

* Stores the decoded text output for:

   * No Rules

   * Rule-Based

   * Boundary-Augmented

The result is a paired comparison dataset of model outputs.

In [ ]:
# ============================================================
# 5. Run the three conditions
# ============================================================
decoded_norules, decoded_rules, decoded_boundaries = [], [], []

for ipa1, ipa2, ipa3 in tqdm(zip(df["ipa_norules"], df["ipa_rules"], df["ipa_boundaries"]),
                             total=len(df), desc="Decoding with T5"):
    decoded_norules.append(decode_ipa(ipa1))
    decoded_rules.append(decode_ipa(ipa2))
    decoded_boundaries.append(decode_ipa(ipa3))

df["t5_norules"] = decoded_norules
df["t5_rules"] = decoded_rules
df["t5_boundaries"] = decoded_boundaries

Decoding with T5: 100%|██████████| 500/500 [03:36<00:00,  2.31it/s]


### **6. Evaluate Model Performance (WER)**

Computes Word Error Rate (WER) for each decoding strategy:

* Quantifies how much the IPA transformation helps or harms decoding.

* Identifies which preprocessing strategy performs best with the baseline model.

This produces the first set of metrics.

In [ ]:
# ============================================================
# 6. Evaluate with WER (or CER if you prefer)
# ============================================================
# if you have text references
if "text_ref" in df.columns:
    ref = df["text_ref"].astype(str)
    df["WER_norules"] = [wer(r, h) for r, h in zip(ref, df["t5_norules"])]
    df["WER_rules"] = [wer(r, h) for r, h in zip(ref, df["t5_rules"])]
    df["WER_boundaries"] = [wer(r, h) for r, h in zip(ref, df["t5_boundaries"])]

    print("Average WERs:")
    print("No rules:", df["WER_norules"].mean())
    print("Rule-based:", df["WER_rules"].mean())
    print("With boundaries:", df["WER_boundaries"].mean())

# Save for inspection
out_path = "/content/drive/MyDrive/Capstone/Corpus/error_atlas/childes_rule_eval.csv"
df.to_csv(out_path, index=False)
print(f"Saved evaluation results to {out_path}")

Average WERs:
No rules: 0.850469432854727
Rule-based: 0.8960444833403037
With boundaries: 0.8695651328662165
Saved evaluation results to /content/drive/MyDrive/Capstone/Corpus/error_atlas/childes_rule_eval.csv


### **7. Load the Fine-Tuned CHILDES Model**

Loads the newly fine-tuned IPA→Text model.

This model was trained on:

* raw IPA

* boundary IPA

* rule-based IPA

It is expected to:

* be more robust to rule-based edits

* better handle boundary tokens

* improve decoding of child IPA overall

This is the core of the second experiment.

In [ ]:
# ============================================================
# 7. Load your fine-tuned T5 model (CHILDES fine-tuned)
# ============================================================
model_id = "zanegraper/t5-ipa-childes-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cpu")

def decode_ipa(ipa_seq):
    inputs = tokenizer(ipa_seq, return_tensors="pt", padding=True, truncation=True).to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=80)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

### **8. Re-run All Three Conditions With Fine-Tuned Model**

Repeats the identical experiment with the fine-tuned model.

This allows direct before-vs-after comparison:

* How much did fine-tuning improve WER?

* Does the fine-tuned model now benefit from boundary cues?

* Do rule-based corrections still hurt, or do they help?

This is the primary experimental result.

In [ ]:
# ============================================================
# 8. Repeat the three conditions
# ============================================================
decoded_norules, decoded_rules, decoded_boundaries = [], [], []

for ipa1, ipa2, ipa3 in tqdm(zip(df["ipa_norules"], df["ipa_rules"], df["ipa_boundaries"]),
                             total=len(df), desc="Decoding with T5"):
    decoded_norules.append(decode_ipa(ipa1))
    decoded_rules.append(decode_ipa(ipa2))
    decoded_boundaries.append(decode_ipa(ipa3))

df["t5_norules"] = decoded_norules
df["t5_rules"] = decoded_rules
df["t5_boundaries"] = decoded_boundaries

Decoding with T5: 100%|██████████| 500/500 [14:13<00:00,  1.71s/it]


### **9. Recompute WER for Fine-Tuned Outputs**

Computes new WER scores for:

* baseline IPA

* rule-based IPA

* boundary-augmented IPA

This reveals:

* Whether fine-tuning shifted performance

* Whether rule-based transformations are now beneficial

* Whether boundaries offer more gain post-training

In [ ]:
# ============================================================
# 9. Re-evaluate with WER (or CER if you prefer)
# ============================================================
# if you have text references
if "text_ref" in df.columns:
    ref = df["text_ref"].astype(str)
    df["WER_norules"] = [wer(r, h) for r, h in zip(ref, df["t5_norules"])]
    df["WER_rules"] = [wer(r, h) for r, h in zip(ref, df["t5_rules"])]
    df["WER_boundaries"] = [wer(r, h) for r, h in zip(ref, df["t5_boundaries"])]

    print("Average WERs:")
    print("No rules:", df["WER_norules"].mean())
    print("Rule-based:", df["WER_rules"].mean())
    print("With boundaries:", df["WER_boundaries"].mean())

# Save for inspection
out_path = "/content/drive/MyDrive/Capstone/Corpus/error_atlas/childes_rule_eval_finetuned.csv"
df.to_csv(out_path, index=False)
print(f"Saved evaluation results to {out_path}")

Average WERs:
No rules: 0.37491888725882533
Rule-based: 0.5323423464507056
With boundaries: 0.4720972510748052
Saved evaluation results to /content/drive/MyDrive/Capstone/Corpus/error_atlas/childes_rule_eval_finetuned.csv


### **Summary of Rule-Based and Boundary-Based IPA Experiments**

This experiment evaluated how three IPA preprocessing strategies impacted the performance of:

1. The original (baseline) IPA→Text T5 model, and

2. The fine-tuned CHILDES T5 model trained on raw, boundary, and rule-based IPA.

The three IPA conditions were:

* No rules — clean IPA only

* Rule-based — IPA with phonological correction rules applied

* With boundaries — IPA augmented with inserted word-boundary markers (`|`)

---

### Baseline Model Results

| Condition       | WER        |
| --------------- | ---------- |
| No rules        | **0.8505** |
| Rule-based      | **0.8960** |
| With boundaries | **0.8696** |

Interpretation

* The baseline model performs poorly overall on CHILDES data (WER ~0.85).

* Rule-based transformations hurt performance even further, likely because the original T5 model was not trained on:

   * repaired IPA sequences

   * epenthetic vowels

   * restored final consonants

* Boundary insertion provides a small improvement but still performs far below satisfactory levels.

**Conclusion:**

The baseline model is highly brittle to any modifications in IPA and is not capable of exploiting rule-based corrections.

---

### Fine-Tuned CHILDES Model Results

| Condition       | WER        |
| --------------- | ---------- |
| No rules        | **0.3749** |
| Rule-based      | **0.5323** |
| With boundaries | **0.4721** |

Interpretation

* Fine-tuning yields a dramatic improvement, reducing WER from ~0.85 to ~0.37 on raw IPA. This represents roughly a 56% reduction in errors.

* However:

   * Rule-based IPA still degrades performance—though less severely than in the baseline run.

   * Boundary insertion now helps, lowering WER from 0.5323 → 0.4721, showing modest but real benefit.

**Conclusion:**
Fine-tuning greatly improves robustness and decoding accuracy, but the model still expects naturalistic IPA rather than manually repaired forms.

Boundary cues are beneficial post-training but rule-based modifications remain harmful.

---

**Overall Findings**

1. Fine-tuning is far more effective than rule-based preprocessing. It cut WER by more than half without needing any IPA manipulation.

2. Rule-based corrections are not yet beneficial for IPA→text decoding. Even with fine-tuning, these corrections introduce unnatural patterns that the model has difficulty interpreting.

3. Boundary markers offer measurable benefits after fine-tuning, suggesting segmentation cues support decoding of child-speech IPA.

4. The strongest-performing pipeline uses:

   * clean IPA without rules,

   * decoded by the fine-tuned T5 model,

   * optionally supplemented with boundary cues if available.

**Takeaway**

Fine-tuning on CHILDES-derived IPA provides substantial improvements in phoneme-to-text decoding.
Rule-based corrections remain counterproductive, while boundary augmentation shows promise but requires more experimentation.
Future work should prioritize larger fine-tuning datasets, boundary-aware training, and possibly data-driven rule learning rather than hand-crafted rules.